In this notebook I update the training set with the results of games in the past. Note that the training set is located at .../splits_optimizer/n=20modelreadydata.csv or some name similar to that. I just want to conform to that, where the shape of that dataset looks like as follow. 


In [1]:
import numpy as np
import pandas as pd
from time import sleep
from random import randint

In [2]:
model_data = pd.read_csv('../splits_optimizer/n=20model_ready_data.csv')
del model_data['Unnamed: 0']

In [3]:
model_data.shape

(4607, 30)

In [4]:
#put this model dataframe in a form for updating/modeling. 
to_update_model_data = model_data  #this will be a dataframe in conjunction to model_data to make my model.
del to_update_model_data['home_SPREAD'],to_update_model_data['PLUS_MINUS'],to_update_model_data['road_SPREAD']

In [5]:
model_data.head(n=3)

,road_Rolling FG,road_Rolling 3P,road_Rolling FT,road_Rolling DR,road_Rolling AST,road_Rolling ST,road_Rolling TO,road_Rolling BL,road_Rolling PTS,road_Rolling AST/TO,...,home_Rolling AST,home_Rolling ST,home_Rolling TO,home_Rolling BL,home_Rolling PTS,home_Rolling AST/TO,home_Rolling 3P%,home_Rolling FG%,home_Rolling FT%,Home Team Won?
0,41.8,9.85,16.5,36.2,27.6,7.45,13.70,5.90,109.95,2.093776,...,25.40,8.9,13.55,3.25,109.65,2.007264,0.325822,0.466812,0.772590,1
1,38.6,8.90,17.4,30.9,22.6,9.00,14.35,4.45,103.50,1.846354,...,22.90,6.1,13.90,4.45,106.25,1.744716,0.428645,0.459249,0.813842,0
2,38.1,8.30,18.7,29.2,23.8,9.35,15.30,5.10,103.20,1.622212,...,20.25,9.8,14.85,5.15,99.55,1.473903,0.301886,0.452531,0.670800,0


In [6]:
to_update_model_data.head()

,road_Rolling FG,road_Rolling 3P,road_Rolling FT,road_Rolling DR,road_Rolling AST,road_Rolling ST,road_Rolling TO,road_Rolling BL,road_Rolling PTS,road_Rolling AST/TO,...,home_Rolling AST,home_Rolling ST,home_Rolling TO,home_Rolling BL,home_Rolling PTS,home_Rolling AST/TO,home_Rolling 3P%,home_Rolling FG%,home_Rolling FT%,Home Team Won?
0,41.80,9.85,16.50,36.20,27.60,7.45,13.70,5.90,109.95,2.093776,...,25.40,8.90,13.55,3.25,109.65,2.007264,0.325822,0.466812,0.772590,1
1,38.60,8.90,17.40,30.90,22.60,9.00,14.35,4.45,103.50,1.846354,...,22.90,6.10,13.90,4.45,106.25,1.744716,0.428645,0.459249,0.813842,0
2,38.10,8.30,18.70,29.20,23.80,9.35,15.30,5.10,103.20,1.622212,...,20.25,9.80,14.85,5.15,99.55,1.473903,0.301886,0.452531,0.670800,0
3,37.85,9.45,16.35,33.45,22.30,7.85,16.80,4.75,101.50,1.489986,...,18.30,6.45,13.15,5.05,89.60,1.544741,0.292427,0.413253,0.699586,1
4,38.30,8.00,20.05,32.40,24.25,7.95,14.05,4.50,104.65,1.830207,...,22.50,8.10,17.55,4.85,102.30,1.324584,0.313071,0.445313,0.685929,0


Same thing! Now to the functions.

I'll be working with a modified form of the functions I created in "predict today" but instead of "today" they'll be scraping arbitrary dates. 

In [7]:
#import dependencies. 
from datetime import datetime
from nba_py import Scoreboard
import numpy as np
import pandas as pd

In [8]:
#get today's date, for the games!


def get_todays_games(month,day,year):
    """
    Get the game's going on today. 
    
    Parameters:
    -----------
    
    None
    
    
    Returns 
    -------
    
    matchups : arr
        array of all the matchups today. Note I skip the first element since it just denotes road, or home which is known
        now. 
        
        
    
    """
    
    
     
    ex = Scoreboard(month=month,day=day,year=year)
    games = ex.game_header()['GAMECODE']
    matchups = []
    matchups.append(['road','home'])
    for game in games:
        away_team = game[-6:-3]
        home_team = game[-3:]

        if away_team == 'BKN':
            away_team = 'BRK'
        if home_team == 'BKN':
            home_team = 'BRK'
        
        matchups.append([away_team,home_team])
        
    return matchups[1:]

In [9]:
def create_data(month,day,year):
    """Creates a dataset identical to the one used for the ML modeling. This is done by scraping the ngames averages
    of the teams just listed, along with the spread, and cominbing. 
    
    Returns 
    -------
    
    today_matchups : arr
        In accordance with the designated format of how these team statistics will be shaped, I did that here. 
        For further explanation, please refer to the "relevant stats" and "splits_optimizer" repo's which explain why I 
        use certain values, this function simply puts them in that shape for the games I want to predict. 
    
    
    home_teams : arr
        Array of the home teams. Since I next have to obtain the spread of these games, I'll line them up based on the 
        name of the home team. 
    """
  #  today = datetime.now()
  #  day = today.day
   # month = today.month
   # year = today.year
    
    
    matchups = get_todays_games(month,day,year)
    print("# of games:",len(matchups))
    if len(matchups) < 1:
        #no games are being played today:
        todays_matchups = None
        home_teams = None
        road_teams = None
        return todays_matchups,home_teams,road_teams
    #matchups
    import numpy as np
    from nba_py import team
    teams = team.TeamList()
    teamids = teams.info()
    #print('predicting matchups of ', teamids)
  #  print()
    teamids = teamids[:-15]
   # print(teamids)
    teamids = teamids[['TEAM_ID','ABBREVIATION']]
    teamids = teamids.rename(index=str, columns={"ABBREVIATION": "Team"})
    teamids = teamids.replace('BKN','BRK')
    teamids = teamids.sort_values('Team')
   # print(teamids)
    todays_matchups = []
    home_teams = []
    road_teams = []
    for matchup in matchups:
        home_teams.append(matchup[1])
        road_teams.append(matchup[0])
        game_array = []
        for team_ in matchup:
            TEAM_ID = teamids.loc[teamids['Team'] == team_].values[0,0]
         #   print(team_,TEAM_ID)
                   ################
            date_ = str(month) +'/'+str(day) +'/'+ str(year)

            TEAM_splits = team.TeamLastNGamesSplits(team_id=TEAM_ID,season='2018-19',date_to=date_)
########## This is the one for updated dates. s
        
        
            TEAM_splits = team.TeamLastNGamesSplits(team_id=TEAM_ID,season='2018-19')
       # print(TEAM_splits.last20())
            df = TEAM_splits.last20()
        
            #retain (and create) the columns proven to be correlated to outcome. 
            df['AST/TO'] = df['AST']/df['TOV']

            df = df[['FGM','FG3M','FTM','DREB','AST','STL',
                     'TOV','BLK','PTS','AST/TO','FG3_PCT',
                     'FG_PCT','FT_PCT']]
           # df['AST/TO'] = df['AST']/df['TOV']
            game_array.append(df.values)
        
        matchup_array = np.concatenate((game_array[0],game_array[1]),axis=1)
        todays_matchups.append(matchup_array)

    #quick formating!
    todays_matchups = np.array(todays_matchups)
    todays_matchups = todays_matchups[:,0,:]
    return todays_matchups,home_teams,road_teams

In [10]:
from datetime import datetime
from nba_py import Scoreboard
import numpy as np
import pandas as pd

def get_winners(month,day,year):
    results = []
    ex = Scoreboard(month=month,day=day,year=year)
    games = ex.line_score()[['TEAM_ABBREVIATION','PTS','GAME_ID']]
    teams = []
    scores = []
    for i, game in enumerate(games.values):
        if game[0] =='BKN':  #naming convention
            game[0] = 'BRK'
        teams.append(game[0])
        scores.append(game[1])
        
    road_teams = teams[0::2]
    home_teams = teams[1::2]
    road_scores = scores[0::2]
    home_scores = scores[1::2]
 #   print("home teams order from the results function:")
 #   print(home_teams)
    for i in range(len(road_teams)):
    
        if road_scores[i] > home_scores[i]:
            results.append(0)
        else:
            results.append(1)
    home_team_order = home_teams
    return home_team_order, results

In [11]:
from datetime import date, timedelta

d1 = date(2018, 11, 29)  # start date
d2 = date(2019,1,25)  # end date

delta = d2 - d1         # timedelta
new_training_data = to_update_model_data.loc[to_update_model_data['home_Rolling DR'] == 600]
for i in range(delta.days + 1):
    sleep(randint(2,7))
    delta_date = d1 + timedelta(i)
   # print("All games that occured on ", delta_date)
   # print(get_todays_games(delta_date.month,delta_date.day,delta_date.year))
    date_ = str(delta_date.month) +'/'+str(delta_date.day) +'/'+ str(delta_date.year)
    print(date_)
    todays_matchups,home_teams,road_teams = create_data(delta_date.month,delta_date.day,delta_date.year)
    if type(todays_matchups) == type(None): #meaning there are games
        print(date_, 'has no games during it.')
        continue #should skip this instance of no games. 
        
    home_team_order, winners = get_winners(delta_date.month,delta_date.day,delta_date.year)
    #print("todays matchups", todays_matchups)
    if home_teams == home_team_order:
        print("The arrangement is still correct, phew!")
        winners = np.array(winners)
        winners = winners.reshape(len(winners),1)
        new_data = np.concatenate((todays_matchups,winners),axis=1)
        for game in new_data:
            new_training_data = new_training_data.append(dict(zip(new_training_data.columns, game)), ignore_index=True)
    else:
        print("Ugh, sorry Noah, gonna have to confirm sorting ")
       # print(home_team_order)
       #print(home_teams)
        break
print('done!')

11/29/2018
# of games: 3
The arrangement is still correct, phew!
11/30/2018
# of games: 11
The arrangement is still correct, phew!
12/1/2018
# of games: 7
The arrangement is still correct, phew!
12/2/2018
# of games: 6
The arrangement is still correct, phew!
12/3/2018
# of games: 7
The arrangement is still correct, phew!
12/4/2018
# of games: 5
The arrangement is still correct, phew!
12/5/2018
# of games: 10
The arrangement is still correct, phew!
12/6/2018
# of games: 3
The arrangement is still correct, phew!
12/7/2018
# of games: 10
The arrangement is still correct, phew!
12/8/2018
# of games: 9
The arrangement is still correct, phew!
12/9/2018
# of games: 4
The arrangement is still correct, phew!
12/10/2018
# of games: 11
The arrangement is still correct, phew!
12/11/2018
# of games: 3
The arrangement is still correct, phew!
12/12/2018
# of games: 11


SSLError: HTTPSConnectionPool(host='stats.nba.com', port=443): Max retries exceeded with url: /stats/teamdashboardbylastngames?DateFrom=&DateTo=12%2F12%2F2018&GameSegment=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlusMinus=N&Rank=N&Season=2018-19&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&TeamID=1610612758&VsConference=&VsDivision= (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')",),))

In [12]:
model_data = pd.concat([to_update_model_data,new_training_data]) #more data for the model now appended. 

In [15]:
y = model_data['Home Team Won?'].values

In [16]:
X = model_data
del X['Home Team Won?']
X = X.values

In [17]:
#X = new_training_data
#del X['Home Team Won?']

array([[ 41.8       ,   9.85      ,  16.5       , ...,   0.32582185,
          0.46681243,   0.7725903 ],
       [ 38.6       ,   8.9       ,  17.4       , ...,   0.42864467,
          0.45924884,   0.81384178],
       [ 38.1       ,   8.3       ,  18.7       , ...,   0.30188582,
          0.45253137,   0.67079982],
       ..., 
       [ 42.3       ,  10.9       ,  17.7       , ...,   0.352     ,
          0.437     ,   0.806     ],
       [ 40.9       ,   9.5       ,  18.8       , ...,   0.427     ,
          0.502     ,   0.794     ],
       [ 40.6       ,  10.9       ,  19.1       , ...,   0.395     ,
          0.476     ,   0.779     ]])

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

X_train, X_test, y_train, y_test = train_test_split(X,y,shuffle=True,test_size=.25)

scaler = MinMaxScaler()

scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [28]:
from sklearn.neural_network import MLPClassifier
from sklearn.cross_validation import cross_val_score


In [29]:
model = MLPClassifier()

model.fit(X_train,y_train)


cvs = cross_val_score(model,X_test,y_test,cv=5)

print("   Model Accuracy: ", 100*np.mean(cvs), "% +/- " , 100 * 2*np.std(cvs), "%. ")


   Model Accuracy:  61.2504985498 % +/-  11.4767538707 %. 


In [ ]:
from sklearn.svm import SVC

model = SVC()
model.fit(X,y)


In [ ]:
model.score(X,y)